<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #19: Process Spyfish Aotearoa videos</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Nov 14th, 2021</h5>

In [ ]:
%load_ext autoreload
%autoreload 2

# Set up and requirements

Import Python packages

In [ ]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t19_utils as t19
#import utils.server_utils as serv_utils

print("Packages loaded successfully")

### Choose your project

In [ ]:
project = t_utils.choose_project()

### Initiate sql and get server or local storage details

In [ ]:
# Initiate db
db_info_dict = t_utils.initiate_db(project.value)

# Select the survey

In [ ]:
import os
import pandas as pd
from ipyfilechooser import FileChooser
from ipywidgets import interactive, Layout

import kso_utils.movie_utils as movie_utils
import kso_utils.server_utils as server_utils
import pandas as pd
import ipywidgets as widgets
import numpy as np
import subprocess
import datetime

def select_survey(db_info_dict):
    # Load the csv with surveys information
    surveys_df = pd.read_csv(db_info_dict["local_surveys_csv"])
    
    # Existing Surveys
    exisiting_surveys = surveys_df.SurveyName.unique()

    def f(Existing_or_new):
        if Existing_or_new == 'Existing':
            survey_widget = widgets.Dropdown(
                options = exisiting_surveys,
                description = 'Survey Name:',
                disabled = False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            display(survey_widget)

            return(survey_widget)

        if Existing_or_new == 'New survey':   
            
            # Load the csv with with sites and survey choices
            choices_df = pd.read_csv(db_info_dict["local_choices_csv"])
            
            # Widget to record the dateEntry of the survey
            DateEntrySurvey_widget = widgets.DatePicker(
                description='Date of entry for the new survey (Today)',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
           
            # Widget to record the encoder of the survey
            EncoderNameSurvey_widget = widgets.Text(
                placeholder='First and last name',
                description='Name of the person encoding this survey:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )

            # Widget to record the start date of the survey
            SurveyStartDate_widget = widgets.DatePicker(
                description='Offical date when survey started as a research event',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )

            # Widget to record the name of the survey
            SurveyName_widget = widgets.Text(
                placeholder='Baited Underwater Video Taputeranga Apr 2015',
                description='A name for this survey:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record the 3 letter abbreviation for the Marine Reserve
            SurveyLocation_widget = widgets.Text(
                placeholder='YYY',
                description='The official 3 letter abbreviation for the Marine Reserve:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            
            # Widget to record the type of survey
            SurveyType_widget = widgets.Dropdown(
                        options = ['BUV','ROV'],
                        value='BUV',
                        description = 'Type of survey:',
                        disabled = False,
                        layout=Layout(width='50%'),
                        style = {'description_width': 'initial'}
            )

            # Widget to record the name of the contractor
            ContractorName_widget = widgets.Text(
                placeholder='No contractor',
                description='Person/company contracted to carry out the survey:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record the number of the contractor
            ContractNumber_widget = widgets.Text(
                description='Contract number for this survey:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record the link to the contract
            LinkToContract_widget = widgets.Text(
                description='Hyperlink to the DOCCM for the contract related to this survey:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record the name of the survey leader
            SurveyLeaderName_widget = widgets.Text(
                placeholder='First and last name',
                description='Name of the person in charge of this survey:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record the name of the linked Marine Reserve
            LinkToMarineReserve_widget = widgets.Dropdown(
                options=choices_df.MarineReserve.unique().tolist(),
                description='Marine Reserve linked to the survey:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record if survey is single species
            FishMultiSpecies_widget = widgets.Dropdown(
                options=["Yes", "No"],
                description='Does this survey look at a single species?',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )

            # Widget to record if survey was stratified by any factor
            StratifiedBy_widget = widgets.Dropdown(
                options=choices_df.Stratification.unique().tolist(),
                description='Stratified factors for the sampling design',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )

            # Widget to record if survey is part of long term monitoring
            IsLongTermMonitoring_widget = widgets.Dropdown(
                options=["Yes", "No"],
                description='Is the survey part of a long-term monitoring?',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )

            # Widget to record the site selection of the survey
            SiteSelectionDesign_widget = widgets.Dropdown(
                options=choices_df.SiteSelection.unique().tolist(),
                description='What was the design for site selection?',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )

            # Widget to record the unit selection of the survey
            UnitSelectionDesign_widget = widgets.Dropdown(
                options=choices_df.UnitSelection.unique().tolist(),
                description='What was the design for site selection?',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record the type of right holder of the survey
            RightsHolder_widget = widgets.Dropdown(
                options=choices_df.RightsHolder.unique().tolist(),
                description='Person(s) or organization(s) owning or managing rights over the resource',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )

            # Widget to record information about who can access the resource
            AccessRights_widget = widgets.Text(
                placeholder='',
                description='Who can access the resource?',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record description of the survey design and objectives
            SurveyVerbatim_widget = widgets.Textarea(
                placeholder='',
                description='Provide an exhaustive description of the survey design and objectives',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )

            # Widget to record the type of BUV
            BUVType_widget = widgets.Dropdown(
                options=choices_df.BUVType.unique().tolist(),
                description='Type of BUV used for the survey:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            
            # Widget to record the type of camera
            CameraModel_widget = widgets.Text(
                placeholder='Make and model',
                description='Describe the type of camera used',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record the camera settings
            CameraSettings_widget = widgets.Text(
                placeholder='Wide lens, 1080x1440',
                description='Describe the camera settings',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )

            # Widget to record the type of bait used
            BaitSpecies_widget = widgets.Text(
                placeholder='Pilchard',
                description='Species that was used as bait for the deployment',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            
            # Widget to record the amount of bait used
            BaitAmount_widget = widgets.BoundedIntText(
                value=500,
                min=200,
                max=1000,
                step=1,
                description='Amount of bait used (g):',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record the link to the pictures
            LinkToPicture_widget = widgets.Text(
                description='Hyperlink to the DOCCM folder for this survey photos:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )

            # Widget to record the name of the vessel
            Vessel_widget = widgets.Text(
                description='Vessel used to deploy the unit:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record the level of the tide
            TideLevel_widget = widgets.Dropdown(
                options=choices_df.TideLevel.unique().tolist(),
                description='Tidal level at the time of sampling:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            
            # Widget to record the weather
            Weather_widget = widgets.Text(
                description='Describe the weather for the survey:',
                disabled=False,
                layout=Layout(width='50%'),
                style = {'description_width': 'initial'}
            )
            


            
            display(DateEntrySurvey_widget,
                    EncoderNameSurvey_widget,
                    SurveyStartDate_widget,
                    SurveyName_widget,
                   SurveyLocation_widget,
                   SurveyType_widget,
                   ContractorName_widget,
                   ContractNumber_widget,
                   LinkToContract_widget,
                   SurveyLeaderName_widget,
                   LinkToMarineReserve_widget,
                   FishMultiSpecies_widget,
                   StratifiedBy_widget,
                   IsLongTermMonitoring_widget,
                   SiteSelectionDesign_widget,
                   UnitSelectionDesign_widget,
                   RightsHolder_widget,
                   AccessRights_widget,
                   SurveyVerbatim_widget,
                   BUVType_widget,
                   CameraModel_widget,
                   CameraSettings_widget,
                   BaitSpecies_widget,
                   BaitAmount_widget,
                   LinkToPicture_widget,
                   Vessel_widget,
                   TideLevel_widget,
                   Weather_widget)

            
            return(DateEntrySurvey_widget,
                  EncoderNameSurvey_widget,
                  SurveyStartDate_widget,
                  SurveyName_widget,
                  SurveyLocation_widget,
                  SurveyType_widget,
                  ContractorName_widget,
                  ContractNumber_widget,
                  LinkToContract_widget,
                  SurveyLeaderName_widget,
                  LinkToMarineReserve_widget,
                  FishMultiSpecies_widget,
                  StratifiedBy_widget,
                  IsLongTermMonitoring_widget,
                  SiteSelectionDesign_widget,
                  UnitSelectionDesign_widget,
                  RightsHolder_widget,
                  AccessRights_widget,
                  SurveyVerbatim_widget,
                  BUVType_widget,
                  CameraModel_widget,
                  CameraSettings_widget,
                  BaitSpecies_widget,
                  BaitAmount_widget,
                  LinkToPicture_widget,
                  Vessel_widget,
                  TideLevel_widget,
                  Weather_widget)

    w = interactive(f,
                    Existing_or_new = widgets.Dropdown(
                        options = ['Existing','New survey'],
                        description = 'Existing or new survey:',
                        disabled = False,
                        layout=Layout(width='50%'),
                        style = {'description_width': 'initial'}
                    )
                   )

    display(w)

    return w
    

In [ ]:
survey_i = select_survey(db_info_dict)

In [ ]:
from ipywidgets import Button, HBox #<----- Add HBox for displaying multiple buttons
import asyncio

def wait_for_change(widget1, widget2): #<------ Rename to widget1, and add widget2
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget1.on_click(getvalue, remove=True) #<------ Rename to widget1
        widget2.on_click(getvalue, remove=True) #<------ New widget2
        # we need to free up the binding to getvalue to avoid an IvalidState error
        # buttons don't support unobserve
        # so use `remove=True` 
    widget1.on_click(getvalue) #<------ Rename to widget1
    widget2.on_click(getvalue) #<------ New widget2
    return future


correct_button = widgets.Button(
          description = 'Yes, details are correct',
          layout=Layout(width='25%'),
          style = {'description_width': 'initial'}
    )
    
wrong_button = widgets.Button(
    description = 'No, I will go back and fix them',
    layout=Layout(width='45%'),
    style = {'description_width': 'initial'}
)


# If new survey, review details and save changes in survey csv server
if isinstance(survey_i.result, tuple):
    # Save the responses as a new row for the survey csv file
    new_survey_row = pd.DataFrame(
        {
            "DateEntrySurvey": [survey_i.result[0].value],
            "EncoderNameSurvey": [survey_i.result[1].value],
            "SurveyStartDate": [survey_i.result[2].value],
            "SurveyName": [survey_i.result[3].value],
            "SurveyLocation": [survey_i.result[4].value],
            "SurveyType": [survey_i.result[5].value],
            "ContractorName": [survey_i.result[6].value],
            "ContractNumber": [survey_i.result[7].value],
            "LinkToContract": [survey_i.result[8].value],
            "SurveyLeaderName": [survey_i.result[9].value],
            "LinkToMarineReserve": [survey_i.result[10].value],
            "FishMultiSpecies": [survey_i.result[11].value],
            "StratifiedBy": [survey_i.result[12].value],
            "IsLongTermMonitoring": [survey_i.result[13].value],
            "SiteSelectionDesign": [survey_i.result[14].value],
            "UnitSelectionDesign": [survey_i.result[15].value],
            "RightsHolder": [survey_i.result[16].value],
            "AccessRights": [survey_i.result[17].value],
            "SurveyVerbatim": [survey_i.result[18].value],
            "BUVType": [survey_i.result[19].value],
            "CameraModel": [survey_i.result[20].value],
            "CameraSettings": [survey_i.result[21].value],
            "BaitSpecies": [survey_i.result[22].value],
            "BaitAmount": [survey_i.result[23].value],
            "LinkToPicture": [survey_i.result[24].value],
            "Vessel": [survey_i.result[25].value],
            "TideLevel": [survey_i.result[26].value],
            "Weather": [survey_i.result[26].value],
        }
    )
    
    # Create new columns for the survey based on the responses
    print(new_survey_row["SurveyStartDate"].values[0])
    new_survey_row["SurveyID"] = new_survey_row["SurveyLocation"]+ "_" + new_survey_row["SurveyStartDate"].values[0].strftime("%Y%m%d") + "_" + new_survey_row["SurveyType"]
    new_survey_row["FishMultiSpecies"] = new_survey_row.replace({'FishMultiSpecies': {'Yes': False, 'No': True}})
    new_survey_row["IsLongTermMonitoring"] = new_survey_row.replace({'IsLongTermMonitoring': {'Yes': True, 'No': False}})
    new_survey_row[["Region","OfficeName","OfficeContact","MarineReserveID"]] = "NA"
    
    print("The details of the new survey are:")
    for ind in new_survey_row.T.index:
        print(ind,"-->", new_survey_row.T[0][ind])
        
    async def f():
        x = await wait_for_change(correct_button,wrong_button) #<---- Pass both buttons into the function
        if x == "Yes, details are correct": #<--- use if statement to trigger different events for the two buttons
            print("The new survey details will be updated.")
        else:
            print("Good job, come back when the data is tidy!")
        

# If existing survey print the info for the pre-existing survey
else:
    # Load the csv with surveys information
    surveys_df = pd.read_csv(db_info_dict["local_surveys_csv"])
    
    # Select the specific survey info
    surveys_df_i = surveys_df[surveys_df["SurveyName"]==survey_i.result.value]
    
    print("The details of the selected survey are:")
    for ind in surveys_df_i.T.index:
        print(ind,"-->", surveys_df_i.T[0][ind])
    
    async def f():
        x = await wait_for_change(correct_button,wrong_button) #<---- Pass both buttons into the function
        if x == "Yes, details are correct": #<--- use if statement to trigger different events for the two buttons
            print("Great, you can start uploading the movies.")
        else:
            print("Good job, come back when the data is tidy!")
        
print("Are the survey details above correct?")
asyncio.create_task(f())
display(HBox([correct_button,wrong_button])) #<----Display both buttons in an HBox
    
    

# Select the folder where the sd card is

Make sure you select the FOLDER not the files

In [ ]:
go_pro_folder = t19.select_go_pro_folder()

In [ ]:
go_pro_movies_i = t19.select_go_pro_movies(go_pro_folder.selected)
print("The movies selected are:")
print(*go_pro_movies_i, sep='\n')

### Select the date and site

In [ ]:
site_i = t19.select_site(db_info_dict)
date_i = t19.select_date()

### Concatenate the go pro files

In [ ]:
video_info_dict = t19.concatenate_go_pro_videos(site_i.result.value,
                                                date_i.value.isoformat().replace("-","_"),
                                                go_pro_folder.selected,
                                                go_pro_movies_i)

### Select Author, bad deployment, survey start/end, s3 bucket

In [ ]:
# Select author
video_author = t19.select_author(db_info_dict)

# Specify id deployment is bad
deployment_i = t19.select_bad_deployment()

# Set survey start
surv_start = t19.select_start_survey(video_info_dict["length"])

#TODO Set survey end
surv_end = t19.select_end_survey(video_info_dict["length"])

# Select the S3 "folder" to upload the video
s3_folder = t19.select_s3_folder(db_info_dict)

#Add any comment related to the movie
comment_movie = t19.write_comment()

### Review the survey/movie details

In [ ]:
new_row, movies_df = t19.review_movie_details(
    project_name = project.value,
    db_info_dict = db_info_dict,
    video_info_dict_i = video_info_dict,
    IsBadDeployment_i = deployment_i.result,
    survey_start_i = surv_start.result,
    survey_end_i = surv_end.result,
    author_i = video_author.result.value,
    comment_i = comment_movie.value,
    s3_prefix_i = s3_folder.value,
    go_pro_list_i = go_pro_movies_i,
    bucket_i = 'marine-buv'
)
print("The new details are:")
new_row.to_dict('records')

# !!!Only pass this point if movie details are correct!!!

### Update movies csv and upload video to s3

In [ ]:
upload_concat_movie(new_row, db_info_dict, video_info_dict, movies_df)

In [ ]:
#END